## Imports

In [1]:
# Import the usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Import Olist data
from olist.data import Olist
from olist.order import Order

# Olist's Net Promoter Score (NPS) 🔥

The **Net Promoter Score (NPS)** of a service answers the following question:

> How likely is it that you would recommend our company/product/service to a friend or colleague?

For a service rated between 1 and 5 stars, like Olist, we can **classify customers into three categories** based on their answers:
- ✅ **Promoters**: customers who answered  with a score of 5
- 😴 **Passive**: customers who answered with a score of 4 
- 😡 **Detractors**: customers who answered with a score between 1 and 3 (inclusive)

<br>

👉 NPS is computed by subtracting the percentage of customers who are **detractors** from the percentage of customers who are **promoters**.

> NPS  
= % Promoters - % Detractors   
= (# Promoter - # Detractors) / # Reviews  
= (# 5 stars - # <4 stars) / # Reviews

## Computing the Overall NPS Score of Olist

In [2]:
data = Olist().get_data()
orders = Order().get_training_data()

In [3]:
orders

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...
96356,9c5dedf39a927c1b2549525ed64a053c,8.218009,18.587442,0.0,delivered,1,0,5,1,1,72.00,13.08
96357,63943bddc261676b46f01ca7ac2f7bd8,22.193727,23.459051,0.0,delivered,0,0,4,1,1,174.90,20.10
96358,83c1379a015df1e13d02aae0204711ab,24.859421,30.384225,0.0,delivered,1,0,5,1,1,205.99,65.02
96359,11c177c8e97725db2631073c19f07b62,17.086424,37.105243,0.0,delivered,0,0,2,2,2,359.98,81.18


❓Create a function that converts `review_score` into `nps_class`. `nps_class` should be a **classification** depending on the `review_score`, so there are 3 possibilities:

- `review_score` is **5** 👉 `nps_class` is **1** (promoter)
- `review_score` is **4** 👉 `nps_class` is **0** (passive)
- `review_score` is **3** or less 👉 `nps_class` is **-1** (detractor)

In [19]:
def promoter_score(review_score):
    if review_score == 5:
        return 1
    elif review_score == 4:
        return 0
    else:
        return -1

nps = orders.review_score.map(promoter_score).mean()

💡Let's try to rewrite this function into a single line of code that achieves the same result 😏

There are **several** ways to do it! Let's look at some of them, then we can compare their execution times to that of our function to see which one is more efficient ⏱️

Two general principles when it comes to programming/coding are:
- `KISS`: **K**eep **I**t **S**imple and **S**mart
- `DRY`: **D**on't **R**epeat **Y**ourself 😉

<details>
    <summary>💡Hint</summary>

Use the following methods and use `%time` to compare their execution times:
- `.apply()` with the function you wrote above
- `.map()` or `.apply()` with a `lambda` function
- `.loc[]` with boolean indexing
- `np.select()` with matching conditions

</details>    

In [16]:
# YOUR CODE HERE
orders.review_score.apply(lambda x: 1 if x == 5 else 0 if x == 4 else -1)

0        0
1        0
2        1
3        1
4        1
        ..
96356    1
96357    0
96358    1
96359   -1
96360    1
Name: review_score, Length: 96353, dtype: int64

**A Note About `.apply()`**

Consider the following examples:

```python
df.apply(lambda col: col.max(), axis = 0)
df.apply(lambda row: row['A'] + row['B'], axis = 1)
```

These operations look similar because they both use `.apply()`, but one is much slower than the other. The data layout for Pandas DataFrames is **column-major** (read more [here](https://en.wikipedia.org/wiki/Row-_and_column-major_order)), which means that column-wise operations are always going to be faster than row-wise operations. The second example above uses `axis=1`, making it a row-wise operation, which would be more appropriate for **row-major** data layouts such as NumPy arrays.

For small amounts of data, this difference is irrelevant, but when you start working with huge datasets this will probably make a big difference. For big datasets, you're likely to notice that using `.loc[]`, `np.select()` or `np.apply_along_axis()` will run faster on Pandas DataFrames when applying a function on every row.

It's always good to understand how your data is stored before you access it!

👇 Now that you have the different promoter scores, you can compute `Olist's NPS`.

In [26]:
# YOUR CODE HERE
f"Olist's NPS equals: {nps*100:.2f}"

"Olist's NPS equals: 38.14"

## NPS per Customer State

👇 Here is the part of Olist's DB schema that is relevant for this section, to help you have an overview of things.

<img src="https://wagon-public-datasets.s3-eu-west-1.amazonaws.com/04-Decision-Science/02-Statistical-Inference/olist_schema.png" width=750>

### What is the average review score per state?

❓First, create the dataset required for computation

In [40]:
# YOUR CODE HERE
merged_df = data["orders"].merge(data["order_reviews"], on="order_id").merge(data["customers"], on="customer_id")
score_per_state = merged_df.groupby("customer_state")["review_score"]
score_per_state.mean().head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

👉 Now, we can aggregate this dataset per  `customer_state` using any aggregation method of our choice :)

❓ Let's start with the average review score: compute the average `review_score` per `customer_state`.

*Hints:* try to tackle this question using three different methods:
- with `.mean()`
- then with `.apply()`
- and eventually the `.agg()`

In [45]:
# YOUR CODE HERE
score_per_state.mean().head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [47]:
# YOUR CODE HERE
score_per_state.apply(np.mean).head()

customer_state
AC    4.049383
AL    3.751208
AM    4.183673
AP    4.194030
BA    3.860888
Name: review_score, dtype: float64

In [49]:
# YOUR CODE HERE
score_per_state.agg({"mean"}).head()

,mean
customer_state,
AC,4.049383
AL,3.751208
AM,4.183673
AP,4.194030
BA,3.860888


🤩 `.agg()` is much more flexible than the other methods, push it further!

In [50]:
# YOUR CODE HERE
score_per_state.agg({"mean"}).head()

,mean
customer_state,
AC,4.049383
AL,3.751208
AM,4.183673
AP,4.194030
BA,3.860888


### NPS per State

❓Now, it is time to create a 🔥 **custom aggregation function** to compute the `NPS per customer_state` directly.

1️⃣ Create your `nps` function

2️⃣ Try to debug it using the `breakpoint()` debugger within your function to understand clearly what objects you are manipulating

<br>

💡 *PS.:* always **cleanly** exit your debugger by typing `exit` when inside the debugging session, otherwise you will have to restart your Notebook!

In [52]:
# YOUR CODE HERE
def nps(x):
   # breakpoint
     return orders.review_score.map(promoter_score).mean()
    

👉 Now, use your `nps` function to compute the `NPS per customer_state`.

In [54]:
# YOUR CODE HERE
merged_df.groupby("customer_state").agg({"review_score": nps}).head()

,review_score
customer_state,
AC,0.381431
AL,0.381431
AM,0.381431
AP,0.381431
BA,0.381431


Again, instead of using this function, try to do the same task in one line of code, remember the `KISS` principle? 😉

In [56]:
# YOUR CODE HERE
merged_df.groupby("customer_state").apply(nps).head()

customer_state
AC    0.381431
AL    0.381431
AM    0.381431
AP    0.381431
BA    0.381431
dtype: float64

# Cheat Sheet for `map`, `apply`, `applymap` and `groupby`

```python
# MAP (for Series)
series.map(function) 
series.map({mapping dict})

# APPLY (for DataFrame)
df.apply(lambda col: col.max(), axis = 0)     # default axis
df.apply(lambda row: row[‘A’] + row[‘B’], axis = 1)

df.applymap(my_funct_for_indiv_elements)
df.applymap(lambda x: '%.2f' % x)
```

```python
## GROUPBY
group = df.groupby('col_A')

group.mean()
group.apply(np.mean)
group.agg({
    col_A: ['mean', np.sum],
    col_B: my_custom_sum,
    col_B: lambda s: my_custom_sum(s)
})

group.apply(custom_mean_function)
```

[Introduction to Pandas' `apply`, `applymap` and `map` - Towards Data Science](https://towardsdatascience.com/introduction-to-pandas-apply-applymap-and-map-5d3e044e93ff)